# IMPORTANDO BIBLIOTECAS NECESSÁRIAS

In [2]:
import pandas as pd

In [3]:
# Considerando a classificação da frase para classificar uma frase em "pergunta" ou "afirmação". Nesse caso, 
# existem duas classes (“pergunta” e “enunciado”). Com o conjunto de treinamento, pode-se treinar um classificador 
# Naive Bayes que podemos usar para categorizar automaticamente uma nova frase.
# O conjunto de treinamento consistirá em quatro perguntas e afirmações. 

In [4]:
columns = ['sent', 'class']
rows = []

rows = [['This is my book', 'stmt'], 
        ['They are novels', 'stmt'],
        ['have you read this book', 'question'],
        ['who is the author', 'question'],
        ['what are the characters', 'question'],
        ['This is how I bought the book', 'stmt'],
        ['I like fictions', 'stmt'],
        ['what is your favorite book', 'question']]

training_data = pd.DataFrame(rows, columns=columns)
training_data

,sent,class
0,This is my book,stmt
1,They are novels,stmt
2,have you read this book,question
3,who is the author,question
4,what are the characters,question
5,This is how I bought the book,stmt
6,I like fictions,stmt
7,what is your favorite book,question


In [5]:
# Agora é necessário descobrir se uma nova frase, digamos, 'qual é o preço do livro' é uma pergunta ou não.
# Usaremos o Teorema Naive de Bayes para resolver isso

In [6]:
# A próxima etapa é apenas calcular todas as probabilidades das equações do teorema acima.

# Para encontrar o número total de vezes que uma palavra aparece em uma classe, será utilizado o CountVectorizer do sklearn.
# CountVectorizer fornece Term-Document Matrix (TDM) para cada classe. Uma matriz de documento de termos (TDM) consiste 
# em uma lista de frequências de palavras que aparecem em um conjunto de documentos.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

stmt_docs = [row['sent'] for index,row in training_data.iterrows() if row['class'] == 'stmt']

vec_s = CountVectorizer()
X_s = vec_s.fit_transform(stmt_docs)
tdm_s = pd.DataFrame(X_s.toarray(), columns=vec_s.get_feature_names())

tdm_s

,are,book,bought,fictions,how,is,like,my,novels,the,they,this
0,0,1,0,0,0,1,0,1,0,0,0,1
1,1,0,0,0,0,0,0,0,1,0,1,0
2,0,1,1,0,1,1,0,0,0,1,0,1
3,0,0,0,1,0,0,1,0,0,0,0,0


In [8]:
# A seguir, calcular o Term-Document Matrix (TDM) para a classe de 'instrução'. 

In [9]:
q_docs = [row['sent'] for index,row in training_data.iterrows() if row['class'] == 'question']

vec_q = CountVectorizer()
X_q = vec_q.fit_transform(q_docs)
tdm_q = pd.DataFrame(X_q.toarray(), columns=vec_q.get_feature_names())

tdm_q

,are,author,book,characters,favorite,have,is,read,the,this,what,who,you,your
0,0,0,1,0,0,1,0,1,0,1,0,0,1,0
1,0,1,0,0,0,0,1,0,1,0,0,1,0,0
2,1,0,0,1,0,0,0,0,1,0,1,0,0,0
3,0,0,1,0,1,0,1,0,0,0,1,0,0,1


In [10]:
# Uma célula da matriz TDM representa a frequência da palavra nessa frase.
# A seguir, deve-secalcular o número total de vezes que cada palavra apareceu em uma classe. 
# Por exemplo, da matriz acima, a palavra ‘livro’ apareceu 2 vezes na classe ‘Pergunta’.

In [11]:
word_list_s = vec_s.get_feature_names();    
count_list_s = X_s.toarray().sum(axis=0) 
freq_s = dict(zip(word_list_s,count_list_s))
freq_s

{'are': 1,
 'book': 2,
 'bought': 1,
 'fictions': 1,
 'how': 1,
 'is': 2,
 'like': 1,
 'my': 1,
 'novels': 1,
 'the': 1,
 'they': 1,
 'this': 2}

In [12]:
# Da mesma forma, frequência de palavras para a classe de perguntas: 

In [13]:
word_list_q = vec_q.get_feature_names();    
count_list_q = X_q.toarray().sum(axis=0) 
freq_q = dict(zip(word_list_q,count_list_q))
freq_q

{'are': 1,
 'author': 1,
 'book': 2,
 'characters': 1,
 'favorite': 1,
 'have': 1,
 'is': 2,
 'read': 1,
 'the': 2,
 'this': 1,
 'what': 2,
 'who': 1,
 'you': 1,
 'your': 1}

In [14]:
# Agora que temos a frequência de cada palavra em cada classe, podemos calcular a probabilidade de cada palavra em 
# uma determinada classe.

In [15]:
prob_s = []
for word, count in zip(word_list_s, count_list_s):
    prob_s.append(count/len(word_list_s))
dict(zip(word_list_s, prob_s))

{'are': 0.08333333333333333,
 'book': 0.16666666666666666,
 'bought': 0.08333333333333333,
 'fictions': 0.08333333333333333,
 'how': 0.08333333333333333,
 'is': 0.16666666666666666,
 'like': 0.08333333333333333,
 'my': 0.08333333333333333,
 'novels': 0.08333333333333333,
 'the': 0.08333333333333333,
 'they': 0.08333333333333333,
 'this': 0.16666666666666666}

In [16]:
# Probabilidades de palavras na classe de perguntas: 

In [25]:
prob_q = []
for count in count_list_q:
    prob_q.append(count/len(word_list_q))
dict(zip(word_list_q, prob_q))

{'are': 0.07142857142857142,
 'author': 0.07142857142857142,
 'book': 0.14285714285714285,
 'characters': 0.07142857142857142,
 'favorite': 0.07142857142857142,
 'have': 0.07142857142857142,
 'is': 0.14285714285714285,
 'read': 0.07142857142857142,
 'the': 0.14285714285714285,
 'this': 0.07142857142857142,
 'what': 0.14285714285714285,
 'who': 0.07142857142857142,
 'you': 0.07142857142857142,
 'your': 0.07142857142857142}

In [18]:
# Agora que sabemos a probabilidade de ocorrência de palavra em uma classe, podemos substituir os valores 

In [19]:
# Visto que a equação acima envolve probabilidades de cada palavra de uma nova frase em relação a uma classe, se uma palavra da 
# nova frase não ocorrer na classe dentro do conjunto de treinamento, a equação torna-se zero. por exemplo: a palavra 'preço' 
# não ocorreu em nosso conjunto de treinamento em nenhuma das classes. Assim, P (preço | Stmt) = 0 e P (preço | Questão) = 0,
# o que anulará a equação. Resumindo, para que a equação acima funcione, é necessário que todas as palavras de cada nova 
# frase apareçam em nosso conjunto de treinamento, o que não é possível.
# Para resolver este problema, usa-se o Laplace Smoothing. 

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

docs = [row['sent'] for index,row in training_data.iterrows()]

vec = CountVectorizer()
X = vec.fit_transform(docs)

total_features = len(vec.get_feature_names())
total_features

21

In [21]:
# Para encontrar, 'contagem total de todos os recursos em uma classe', 

In [22]:
total_cnts_features_s = count_list_s.sum(axis=0)
total_cnts_features_q = count_list_q.sum(axis=0)

In [23]:
# Probabilidade com suavização laplace para todas as palavras na classe de instrução 

In [26]:
prob_s_with_ls = []
for word in new_word_list:
    if word in freq_s.keys():
        count = freq_s[word]
    else:
        count = 0
    prob_s_with_ls.append((count + 1)/(total_cnts_features_s + total_features))
dict(zip(new_word_list,prob_s_with_ls))

NameError: name 'new_word_list' is not defined